<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from collections import Counter
import random
import time
import math

def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def obtener_fpm_dict(df_fpm):
    df_fpm_fpm = df_fpm[df_fpm['Tema'] == 'FPM']
    return dict(zip(df_fpm_fpm['Datos'], df_fpm_fpm['Valor']))

def calcular_TVD(dict_d1, dict_d2): # Distancia Total de Variación.
    soporte_total = set(dict_d1) | set(dict_d2)
    return 0.5 * sum(abs(dict_d1.get(k, 0) - dict_d2.get(k, 0)) for k in soporte_total)

# --------------------------- Pandas ---------------------------
def comparar_dfs(df_A, df_B):
    df_comparado = pd.merge(df_A[['Tema', 'Datos', 'Valor']],
                            df_B[['Datos', 'Valor']],
                            on='Datos',
                            how='outer',
                            suffixes=('_A', '_B'))

    df_comparado['Diferencia'] = (df_comparado['Valor_A'] - df_comparado['Valor_B']).abs()
    return df_comparado[['Tema', 'Datos', 'Valor_A', 'Valor_B', 'Diferencia']]

def crear_df_fpm(dict_fpm):
    soporte = list(dict_fpm.keys())
    data = [{
        'Tema': 'FPM',
        "Datos": x,
        "Valor": dict_fpm.get(x, 0)
    } for x in soporte]

    esp = sum(x * p for x, p in dict_fpm.items())
    var = sum((x - esp)**2 * p for x, p in dict_fpm.items())
    desv = var**0.5
    resumen = [
        {'Tema': 'Estadisticas', "Datos": "Esperanza", "Valor": esp},
        {'Tema': 'Estadisticas', "Datos": "Varianza", "Valor": var},
        {'Tema': 'Estadisticas', "Datos": "Desv. Est.", "Valor": desv},
    ]

    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def estimar_df_fpm(generador_de_muestras, n=100_000):
    muestras, tiempos = zip(*(medir_tiempo(generador_de_muestras) for _ in range(n)))

    frec = Counter(muestras)
    soporte = sorted(list(frec.keys()))

    dict_fpm_estimada = {x: frec.get(x, 0) / n for x in soporte}
    df_fpm_estimada = crear_df_fpm(dict_fpm_estimada)

    tiempo_promedio = sum(tiempos) / len(tiempos)
    df_tiempo_promedio = pd.DataFrame([{
        'Tema': 'Performance',
        "Datos": "Tiempo Promedio",
        "Valor": tiempo_promedio
    }])

    return pd.concat([df_fpm_estimada, df_tiempo_promedio], ignore_index=True)

# --------------------------- Utilidades ---------------------------

def ordenar_dict_fpm_por_probabilidad(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  dict_fda = {}
  acc = 0
  for v, p in dict_fpm.items():
    acc += p
    dict_fda[v] = acc
  return dict_fda

# def crear_dict_fpm(soporte, fpm):
#   return {v: fpm(v) for v in soporte}

# dict_A = dict(gen_A)
# gen_A = iter(dict_A.items())

def probabilidad(condicion, n):
    exitos = sum(condicion() for _ in range(n))
    return exitos / n

def gen_fpm_truncada(gen_fpm, condicion):
  list_fpm = list(gen_fpm())
  suma_probabilidades = sum(p for i, p in list_fpm if condicion(i))

  for i, p in list_fpm:
    if condicion(i):
      yield i, p / suma_probabilidades

def gen_fpm(gen_soporte, fpm):
  soporte = gen_soporte()
  fda_v = 0
  while fda_v < 1 - 1e-15:
    v = next(soporte)
    fpm_v = fpm(v)
    yield v, fpm_v
    fda_v += fpm_v

def gen_fda(gen_soporte, fpm):
  soporte = gen_soporte()
  fda_v = 0
  while fda_v < 1 - 1e-15:
    v = next(soporte)
    fda_v += fpm(v)
    yield v, fda_v

def TI(gen_soporte, fpm):
  u = random.random()
  soporte = gen_soporte()
  fda_v = 0
  for v in soporte:
    fda_v += fpm(v)
    if u <= fda_v:
      return v

# dict_A = dict(gen_A)
# def TI(dict_fda):
#   u = random.random()
#   for v, p in dict_fda.items():
#     if u < p:
#       return v

def AyR(dict_fpm_objetivo, fpm_propuesta, dict_fpm_propuesta, C):
    while True:
        muestra = fpm_propuesta()
        prob_objetivo = dict_fpm_objetivo.get(muestra, 0)
        prob_propuesta = dict_fpm_propuesta.get(muestra, 0)

        if prob_propuesta == 0:
            continue

        if random.random() < prob_objetivo / (C * prob_propuesta):
            return muestra

In [9]:
import numpy as np
P1 = 0.05; P2 = 0.2

# generador de muestras del ejercicio:
def QueDevuelve(p1,p2):
  u1 = random.random()
  u2 = random.random()
  X = int(np.log(1-u1)/np.log(1-p1))+1
  Y = int(np.log(1-u2)/np.log(1-p2))+1
  return min(X,Y)

df_fpm_QueDevuelve = estimar_df_fpm(lambda: QueDevuelve(P1, P2))

## Resolucion 1) generador de muestras (para dos probabilidades, p1 y p2) con la misma
# distribucion que min(Geom(p1), Geom(p2))
def minXY_Geometrica(p1,p2):
  u = random.random()
  return int(np.log(1-u) / np.log((1-p1) * (1-p2))) + 1

df_fpm_minXY_Geometrica = estimar_df_fpm(lambda: minXY_Geometrica(P1, P2))

## Resolucion 2) generador de muestras normal con distribucion Geom(p)
def X_Geometrica(p):
    u = random.random()
    return int(np.log(1 - u) / np.log(1 - p)) + 1

# un generador de muestras normal con p = P1 + P2 - P1 * P2 tiene distribucion
# min(Geom(P1), Geom(P2))
p_min = P1 + P2 - P1 * P2
df_fpm_X_Geometrica = estimar_df_fpm(lambda: X_Geometrica(p_min))

## Comparaciones:
print(calcular_TVD(obtener_fpm_dict(df_fpm_QueDevuelve), obtener_fpm_dict(df_fpm_minXY_Geometrica)))
comparar_dfs(df_fpm_QueDevuelve, df_fpm_X_Geometrica)

0.006069999999999994


,Tema,Datos,Valor_A,Valor_B,Diferencia
0,FPM,1,0.239900,0.240340,0.000440
1,FPM,2,0.182510,0.182730,0.000220
2,FPM,3,0.139550,0.136470,0.003080
3,FPM,4,0.105210,0.107580,0.002370
4,FPM,5,0.080550,0.079130,0.001420
5,FPM,6,0.061040,0.060150,0.000890
6,FPM,7,0.045210,0.046390,0.001180
7,FPM,8,0.034500,0.035510,0.001010
8,FPM,9,0.027800,0.026630,0.001170
9,FPM,10,0.020030,0.020920,0.000890
